In [1]:
from typing import Annotated, Literal, TypedDict, List
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from pinecone import Pinecone, ServerlessSpec

from difflib import get_close_matches
import json
import os
from dotenv import load_dotenv
load_dotenv()

True

In [159]:
INDEX_NAME = "funeral-services"
pc = Pinecone()
index = pc.Index(INDEX_NAME)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
print(index.describe_index_stats(), '\n\n')

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'funeral_facilities': {'vector_count': 2518},
                'funeral_facilities_with_gaslighting': {'vector_count': 2518},
                'guide': {'vector_count': 4},
                'ordinance': {'vector_count': 234}},
 'total_vector_count': 5274,
 'vector_type': 'dense'} 




In [160]:
vectorstore_ordinance = PineconeVectorStore(
    index=index, 
    embedding=embeddings,
    namespace="ordinance"
)
vectorstore_funeral_facilities = PineconeVectorStore(
    index=index, 
    embedding=embeddings,
    namespace='funeral_facilities'
)
# vectorstore_funeral_facilities_with_gaslighting = PineconeVectorStore(
#     index=index, 
#     embedding=embeddings,
#     namespace='funeral_facilities_with_gaslighting'
# )

In [4]:
with open('../data/processed/region_list.json', 'r', encoding='utf-8') as f:
    region_list_json = json.load(f)
    print(region_list_json)

{'public_funeral_ordinance': ['가평군', '강화군', '경기도', '고양시', '과천시', '광명시', '광주시', '구리시', '김포시', '남동구', '남양주시', '동두천시', '부천시', '서울특별시', '서울특별시 강남구', '서울특별시 강동구', '서울특별시 강북구', '서울특별시 강서구', '서울특별시 관악구', '서울특별시 광진구', '서울특별시 구로구', '서울특별시 금천구', '서울특별시 노원구', '서울특별시 도봉구', '서울특별시 동대문구', '서울특별시 동작구', '서울특별시 마포구', '서울특별시 서대문구', '서울특별시 서초구', '서울특별시 성동구', '서울특별시 성북구', '서울특별시 양천구', '서울특별시 영등포구', '서울특별시 용산구', '서울특별시 은평구', '서울특별시 종로구', '서울특별시 중구', '서울특별시 중랑구', '성남시', '수원시', '시흥시', '안산시', '안성시', '안양시', '양주시', '양평군', '여주시', '연천군', '오산시', '옹진군', '용인시', '의왕시', '의정부시', '이천시', '인천광역시', '인천광역시 계양구', '인천광역시 동구', '인천광역시 미추홀구', '인천광역시 부평구', '인천광역시 서구', '인천광역시 연수구', '인천광역시 중구', '파주시', '포천시', '화성시'], 'cremation_words_support': [], 'cremation_detail': ['강원도 고성군', '강원도 삼척시', '강원도 양구군', '강원도 양양군', '강원도 영월군', '강원도 철원군', '강원도 평창군', '강원도 화천군', '경기도 가평군', '경기도 과천시', '경기도 광주시', '경기도 구리시', '경기도 김포시', '경기도 남양주시', '경기도 안성시', '경기도 양주시', '경기도 양평군', '경기도 연천군', '경기도 오산시', '경기도 의왕시', '경기도 이천시', '경기도 평택시', '경기도 하남시', '경남 거창군', '경남 산

In [167]:
with open('../data/processed/facilities_region_list.json', 'r', encoding='utf-8') as f:
    facilities_region_list_json = json.load(f)
    print(facilities_region_list_json)

{'묘지': ['강원도 태백시', '강원특별자치도 강릉시', '강원특별자치도 고성군', '강원특별자치도 동해시', '강원특별자치도 삼척시', '강원특별자치도 속초시', '강원특별자치도 양구군', '강원특별자치도 양양군', '강원특별자치도 영월군', '강원특별자치도 원주시', '강원특별자치도 인제군', '강원특별자치도 정선군', '강원특별자치도 철원군', '강원특별자치도 춘천시', '강원특별자치도 평창군', '강원특별자치도 홍천군', '강원특별자치도 화천군', '강원특별자치도 횡성군', '경기 파주시', '경기도 가평군', '경기도 고양시', '경기도 광주시', '경기도 구리시', '경기도 김포시', '경기도 남양주시', '경기도 동두천시', '경기도 성남시', '경기도 시흥시', '경기도 안산시', '경기도 안성시', '경기도 양주시', '경기도 양평군', '경기도 여주시', '경기도 연천군', '경기도 오산시', '경기도 용인시', '경기도 의왕시', '경기도 의정부시', '경기도 이천시', '경기도 파주시', '경기도 평택시', '경기도 포천시', '경기도 화성시', '경상남도 거제시', '경상남도 거창군', '경상남도 고성군', '경상남도 김해시', '경상남도 남해군', '경상남도 사천시', '경상남도 양산시', '경상남도 의령군', '경상남도 진주시', '경상남도 창원시', '경상남도 함안군', '경상남도 함양군', '경상북도 경산시', '경상북도 경주시', '경상북도 구미시', '경상북도 김천시', '경상북도 상주시', '경상북도 성주군', '경상북도 안동시', '경상북도 영덕군', '경상북도 울진군', '경상북도 칠곡군', '경상북도 포항시', '광주광역시 남구', '광주광역시 북구', '대구광역시 군위군', '대구광역시 달서구', '대구광역시 달성군', '대구광역시 북구', '대구광역시 수성구', '대구광역시 중구', '대전광역시 동구', '대전광역시 서구', '부산광역시 금정구', '부산광역시 기장군', '부산광역시 남구', '서울특별시 도봉구'

In [6]:
def find_matching_type(user_input, type_list):
    # 1. 양방향 체크
    for type in type_list:
        if user_input in type or type in user_input:
            return type
    
    # 2. 유사도 기반 매칭
    matches = get_close_matches(user_input, type_list, n=1, cutoff=0.6)
    return matches[0] if matches else None

In [168]:
def find_matching_regions(user_input, region_list, n=3):
    """유사한 지역 여러 개 반환"""
    matched = []
    
    # 1. 양방향 체크
    for region in region_list:
        if user_input in region or region in user_input:
            matched.append(region)
            if len(matched) >= n:
                return matched
    
    # 2. 유사도 기반 매칭
    if not matched:
        matched = get_close_matches(user_input, region_list, n=n, cutoff=0.6)
    
    return matched if matched else None

In [8]:
find_matching_regions("서울", region_list = region_list_json["public_funeral_ordinance"])

['서울특별시', '서울특별시 강남구', '서울특별시 강동구']

In [158]:
list(facilities_region_list_json.values())

[['강원도 태백시',
  '강원특별자치도 강릉시',
  '강원특별자치도 고성군',
  '강원특별자치도 동해시',
  '강원특별자치도 삼척시',
  '강원특별자치도 속초시',
  '강원특별자치도 양구군',
  '강원특별자치도 양양군',
  '강원특별자치도 영월군',
  '강원특별자치도 원주시',
  '강원특별자치도 인제군',
  '강원특별자치도 정선군',
  '강원특별자치도 철원군',
  '강원특별자치도 춘천시',
  '강원특별자치도 평창군',
  '강원특별자치도 홍천군',
  '강원특별자치도 화천군',
  '강원특별자치도 횡성군',
  '경기 파주시',
  '경기도 가평군',
  '경기도 고양시',
  '경기도 광주시',
  '경기도 구리시',
  '경기도 김포시',
  '경기도 남양주시',
  '경기도 동두천시',
  '경기도 성남시',
  '경기도 시흥시',
  '경기도 안산시',
  '경기도 안성시',
  '경기도 양주시',
  '경기도 양평군',
  '경기도 여주시',
  '경기도 연천군',
  '경기도 오산시',
  '경기도 용인시',
  '경기도 의왕시',
  '경기도 의정부시',
  '경기도 이천시',
  '경기도 파주시',
  '경기도 평택시',
  '경기도 포천시',
  '경기도 화성시',
  '경상남도 거제시',
  '경상남도 거창군',
  '경상남도 고성군',
  '경상남도 김해시',
  '경상남도 남해군',
  '경상남도 사천시',
  '경상남도 양산시',
  '경상남도 의령군',
  '경상남도 진주시',
  '경상남도 창원시',
  '경상남도 함안군',
  '경상남도 함양군',
  '경상북도 경산시',
  '경상북도 경주시',
  '경상북도 구미시',
  '경상북도 김천시',
  '경상북도 상주시',
  '경상북도 성주군',
  '경상북도 안동시',
  '경상북도 영덕군',
  '경상북도 울진군',
  '경상북도 칠곡군',
  '경상북도 포항시',
  '광주광역시 남구',
  '광주광역시 북구',
  '대구광역시 군위군',
  '대구광

In [9]:
@tool
def search_public_funeral_ordinance(query: str, region: str = None):
    """
    공영장례 조례를 검색합니다.
    
    Args:
        query: 검색어 (예: "지원 대상")
        region: 지역명 (예: "수원시", "서울특별시 강남구, 인천광역시 서구")
    """
    # filter_dict 먼저 초기화
    filter_dict = {"type": "Public_Funeral_Ordinance"}
    k = 3
    
    if region:
        region_list = region_list_json["public_funeral_ordinance"]
        matched = find_matching_regions(region, region_list, n=k)  # 여러 개
        print(f"공영 장례 조례 매칭된 지역들 최대 {k}개",matched)
        if matched:
            if len(matched) == 1:
                filter_dict["region"] = matched[0]  # 1개면 직접
            else:
                filter_dict["region"] = {"$in": matched}  # 여러 개면 in

    results = vectorstore_ordinance.similarity_search(query, k=k, filter=filter_dict)
    # print(f"공영 장례 조례 검색된 문서 개수: {len(results)}")  # 추가
    # for i, doc in enumerate(results):
    #     print(f"{i+1}. {doc.metadata.get('region')}")  # 지역 확인
    print("툴 검색 결과:",results)
    return results

In [10]:
@tool
def search_cremation_subsidy_ordinance(query: str, region: str = None):
    """
    화장 장려금 조례를 검색합니다.  
    제외 대상에 대한 정보가 말이 뒤죽박죽 되어 이해하기 어려울 경우 다음의 사항을 바탕으로 이해한다.
    1.「장사 등에 관한 법률」 제7조 제2항을 위반한 경우
    2. 다른 법령에 따라 화장에 대한 지원금을 받은 경우
    Args:
        query: 검색어 (예: "지원 대상")
        region: 지역명 (예: "강원도 고성군", "서울 강남")
    """
    filter_dict = {"type": "Cremation_Subsidy_Ordinance"}
    k = 3
    
    if region:
        region_list = region_list_json["cremation_detail"] + region_list_json["cremation_etcetera"]
        matched = find_matching_regions(region, region_list, n=k)  # 여러 개
        print(f"화장 장려금 조례 매칭된 지역들 최대 {k}개",matched)

        if matched:
            if len(matched) == 1:
                filter_dict["region"] = matched[0]  # 1개면 직접
            else:
                filter_dict["region"] = {"$in": matched}  # 여러 개면 in

    results = vectorstore_ordinance.similarity_search(query, k=k, filter=filter_dict)
    
    print("툴 검색 결과:",results)
    return results

In [171]:
all_regions = []

for r_list in facilities_region_list_json.values():
        all_regions.extend(r_list)
len(all_regions)

712

In [ ]:
import traceback
import numpy as np
from langchain_community.utils.math import cosine_similarity

@tool
def search_funeral_facilities_re_search(query: str, region: str = None):
    """
    장례 시설을 검색합니다.
    1차로 지역명(없으면 쿼리)으로 넓게 검색하고, 2차로 상세 쿼리와의 유사도를 비교하여 최적의 결과를 반환합니다.
    
    Args:
        query: 검색 쿼리 (예: "포천시 천주교 관련 묘지")
        region: 지역명 (예: "경기도 포천시")
    """
    print(f"입력된 쿼리: {query} / 지역: {region}")

    try:
        # [Step 1] 1차 검색: 지역명으로 100개 후보군 확보 (Broad Search)
        # 지역명이 있으면 지역명으로, 없으면 쿼리 자체로 넓게 검색
        search_keyword = region if region else query
        
        candidates = vectorstore_funeral_facilities.similarity_search(
            search_keyword,
            k=100
        )
        
        if not candidates:
            print("1차 검색 결과 없음")
            return "해당 조건에 맞는 시설을 찾을 수 없습니다."

        
        # 1. 사용자의 질문(Query) 임베딩 생성
        query_embedding = embeddings.embed_query(query)
        
        # 2. 1차에서 찾은 후보군 문서들의 텍스트 임베딩 생성
        candidate_texts = [doc.page_content for doc in candidates]
        candidate_embeddings = embeddings.embed_documents(candidate_texts)

        # 3. 코사인 유사도 계산 (Query vs Candidates)
        scores = cosine_similarity([query_embedding], candidate_embeddings)[0]
        
        # 4. 점수가 높은 순서대로 정렬하여 상위 5개 선택
        top_k_indices = np.argsort(scores)[::-1][:5]
        
        final_results = [candidates[i] for i in top_k_indices]
        
        print(f"최종 검색 결과 {len(final_results)}건 반환")
        return final_results

    except Exception as e:
        print(traceback.format_exc())
        return f"검색 중 오류 발생: {str(e)}"

In [200]:
@tool
def search_funeral_facilities(query: str, region: str = None, regions : List[str] = None):
    """
    장례 시설을 검색합니다. 
    
    Args:
        query: 검색 문장 (예: "경기도 수원시 시설 좋은 묘지", "대구 남구 천주교 납골당")
        region: 지역명, 지역 한 개 검색 시 사용 (예: "경기도 성남시", "경상북도 경주시")
        regions: 지역명, 지역 여러개 검색 시 사용 (예: ["경기도 의왕시", "경기도 안양시", "경기도 군포시"], ["경상남도 양산시", "경상남도 밀양시"])

    """
    print(f"쿼리 : {query}, 지역 : {region}, 지역들 : {regions}")

    all_regions = []
    for r_list in facilities_region_list_json.values():
        all_regions.extend(r_list)
    all_regions = sorted(list(set(all_regions)))

    results_list = []
    if regions == None:
        regions = [region]
    k = 10 // len(regions) 

    for rgn in regions:

        filter_dict = {}

        if rgn: # 하나의 지역
            matched = find_matching_regions(rgn, all_regions, n=100) 
            print(f"매칭된 지역: {matched}")

            if matched:
                if len(matched) == 1:
                    filter_dict["region"] = matched[0]  
                else:
                    filter_dict["region"] = {"$in": matched} 

        try:
            results = vectorstore_funeral_facilities.similarity_search(
                query, 
                k=k, 
                filter=filter_dict
            )
            print(f"검색 결과 {len(results)}건 반환")
            results_list.extend(results)
            
        except Exception as e:
            print(f"검색 오류: {e}")
            continue
        
        
    unique_results = []
    seen_content = set()
    for doc in results_list:
        if doc.page_content not in seen_content:
            unique_results.append(doc)
            seen_content.add(doc.page_content)

    print(f"최종 병합된 검색 결과: {len(unique_results)}건")
    return unique_results

In [152]:
(list(facilities_region_list_json.values()))

[['강원도 태백시',
  '강원특별자치도 강릉시',
  '강원특별자치도 고성군',
  '강원특별자치도 동해시',
  '강원특별자치도 삼척시',
  '강원특별자치도 속초시',
  '강원특별자치도 양구군',
  '강원특별자치도 양양군',
  '강원특별자치도 영월군',
  '강원특별자치도 원주시',
  '강원특별자치도 인제군',
  '강원특별자치도 정선군',
  '강원특별자치도 철원군',
  '강원특별자치도 춘천시',
  '강원특별자치도 평창군',
  '강원특별자치도 홍천군',
  '강원특별자치도 화천군',
  '강원특별자치도 횡성군',
  '경기 파주시',
  '경기도 가평군',
  '경기도 고양시',
  '경기도 광주시',
  '경기도 구리시',
  '경기도 김포시',
  '경기도 남양주시',
  '경기도 동두천시',
  '경기도 성남시',
  '경기도 시흥시',
  '경기도 안산시',
  '경기도 안성시',
  '경기도 양주시',
  '경기도 양평군',
  '경기도 여주시',
  '경기도 연천군',
  '경기도 오산시',
  '경기도 용인시',
  '경기도 의왕시',
  '경기도 의정부시',
  '경기도 이천시',
  '경기도 파주시',
  '경기도 평택시',
  '경기도 포천시',
  '경기도 화성시',
  '경상남도 거제시',
  '경상남도 거창군',
  '경상남도 고성군',
  '경상남도 김해시',
  '경상남도 남해군',
  '경상남도 사천시',
  '경상남도 양산시',
  '경상남도 의령군',
  '경상남도 진주시',
  '경상남도 창원시',
  '경상남도 함안군',
  '경상남도 함양군',
  '경상북도 경산시',
  '경상북도 경주시',
  '경상북도 구미시',
  '경상북도 김천시',
  '경상북도 상주시',
  '경상북도 성주군',
  '경상북도 안동시',
  '경상북도 영덕군',
  '경상북도 울진군',
  '경상북도 칠곡군',
  '경상북도 포항시',
  '광주광역시 남구',
  '광주광역시 북구',
  '대구광역시 군위군',
  '대구광

In [ ]:
import traceback
from typing import List

@tool
def search_funeral_facilities_hard_filter(query: List[str], region: str = None):
    """
    장례 시설을 검색합니다. 묘지, 봉안당, 화장장, 자연장지, 장례식장 중 하나 또는 여러개를 검색합니다. 
    
    Args:
        query: 검색 쿼리 리스트 (예: ["묘지"], ["봉안당", "화장장"])
        region: 지역명 (예: "경기도 성남시", "강원도 동해시")
    """
    print(f"입력된 쿼리 리스트: {query}")

    # 입력된 쿼리 각각에 대해 시설 유형 매칭
    korean_facilities_type = []
    
    facility_map = {
        "묘지": "cemetery",
        "봉안당": "columbarium",
        "납골당": "columbarium",   
        "화장장": "crematorium",
        "자연장지": "natural_burial",
        "장례식장": "funeral_home"
    }    
    for q in query:
        korean_facilities_type.append(find_matching_type(q, list(facility_map.keys())))
    

    results_list = []
    for kor_fac in korean_facilities_type: 
        # 매칭된 시설 타입이 없으면 건너뜀
        if kor_fac is None:
            continue
        facility = facility_map[kor_fac]
        try:
            filter_dict = {}
            
            # 지역 필터링 적용
            if region:
                region_list = facilities_region_list_json[facility]
                matched = find_matching_regions(region, region_list, n=2)
                print(f"[{facility}] 매칭된 지역: {matched}")
                
                if matched:
                    if len(matched) == 1:
                        filter_dict["full_region"] = matched[0]
                    else:
                        filter_dict["full_region"] = {"$in": matched}
            
            
            results = vectorstore_funeral_facilities.similarity_search(
                facility, 
                k=2, 
                filter=filter_dict
            )
            results_list.extend(results)
            
        except Exception as e:
            print(f"[{facility}] 검색 중 에러 발생: {e}")
            traceback.print_exc()

    return results_list

In [201]:
# --- LangGraph 상태 및 노드 정의
tools = [search_public_funeral_ordinance, 
         search_cremation_subsidy_ordinance, 
        #  search_funeral_facilities,
        #  search_funeral_facilities_sim,
         search_funeral_facilities
]

# LLM 모델 로드 및 도구 바인딩
llm = ChatOpenAI(model="gpt-4o", temperature=0.1)
llm_with_tools = llm.bind_tools(tools)

# 그래프의 State 정의 (대화 기록 유지)
class AgentState(TypedDict):
    messages: Annotated[list, add_messages]

def agent_node(state: AgentState):
    # 현재까지의 대화 기록을 가져옴
    current_messages = state['messages']
    
    # 시스템 프롬프트 정의
    system_prompt = SystemMessage(content="""
    당신은 삶의 마지막을 준비하거나 소중한 이를 떠나보내는 분들을 돕는 따뜻하고 사려 깊은 '웰다잉 도우미' 챗봇입니다.

    [도구 사용 및 지역 처리 핵심 지침]

    1. **시설 검색 (`search_funeral_facilities`) 규칙:**
    - **목적:** 장례식장, 봉안당, 묘지, 화장장 등의 위치를 찾을 때 사용합니다.
    - **복수 지역 허용:** 사용자가 '수도권', '서울 근교', '서울과 수원 사이' 등 넓은 범위를 말하면, 관련된 시/군/구를 추론하여 `regions` 인자에 **리스트 형태**로 **최대3개** 입력하세요.
        (예: `regions=["안양시", "의왕시", "군포시"]`)
    - 한 지역 검색: 한 지역을 검색할 경우 region인자에 string타입으로 넣어줘서 지역을 찾습니다. (예: `region ="충남"`)

    2. **조례/지원금 검색 (`search_public_funeral_ordinance`, `search_cremation_subsidy_ordinance`) 규칙:**
    - **목적:** 공영장례 지원이나 화장 장려금 등 지자체 행정 지원을 찾을 때 사용합니다.
    - **단일 지역 필수:** 지원금은 **'고인의 주민등록상 거주지'**가 기준이므로, 반드시 **하나의 명확한 지역명(문자열)**만 `region` 인자에 입력해야 합니다.
    - **모호한 경우:** 만약 사용자가 "수도권의 화장장려금 알려줘"라고 묻는다면, 리스트로 검색하지 말고 **"정확한 안내를 위해 거주하시는 시/군/구를 말씀해 주시겠어요?"**라고 되물어보세요.

    3. **공통 대화 태도:**
    - 사용자가 구체적인 지역을 언급하지 않았다면, 먼저 정중하게 지역을 확인하세요.
    - 검색 결과가 없을 경우, 솔직하게 해당 지역의 정보가 없음을 알리고 인근 지역이나 대안을 제시해 주세요.
    - 항상 따뜻한 위로와 공감의 말을 잊지 마세요.
    """)

    # 대화 기록 맨 앞에 시스템 메시지가 없으면, 이번 턴(invoke)에만 임시로 붙여서 보냄
    # (state에 영구 저장하지 않음으로써 중복 방지)
    if not isinstance(current_messages[0], SystemMessage):
        messages_to_send = [system_prompt] + current_messages
    else:
        messages_to_send = current_messages

    # LLM 호출
    response = llm_with_tools.invoke(messages_to_send)
    # print("response:", response)
    
    # 새로운 응답만 리스트에 담아 반환 (add_messages가 알아서 기존 기록 뒤에 붙여줌)
    return {"messages": [response]}

# [Node 2] 도구 실행 노드
tool_node = ToolNode(tools)

# [Edge] 조건부 엣지: 도구를 쓸지, 답변을 끝낼지 결정
def should_continue(state: AgentState) -> Literal["tools", "__end__"]:
    messages = state['messages']
    last_message = messages[-1]

    print(f"Tool Calls 존재 여부: {last_message.tool_calls}")
    
    # LLM이 도구 호출을 요청했는지 확인
    if last_message.tool_calls:
        return "tools"
    return "__end__"

In [202]:
# --- 그래프 구성 (Workflow)
workflow = StateGraph(AgentState)

# 노드 추가
workflow.add_node("agent", agent_node)
workflow.add_node("tools", tool_node)

# 엣지 연결
workflow.set_entry_point("agent") # 시작점
workflow.add_conditional_edges(
    "agent",
    should_continue,
)
workflow.add_edge("tools", "agent") # 도구 실행 후 다시 에이전트로 복귀 (결과 해석)

# 그래프 컴파일
app = workflow.compile()

In [203]:
# --- 실행 테스트
def chat(user_input):
    print(f"\n사용자: {user_input}")
    inputs = {"messages": [HumanMessage(content=user_input)]}
    
    # 그래프 실행 (스트리밍으로 과정 확인 가능)
    for event in app.stream(inputs):
        for key, value in event.items():
            if key == "agent":
                msg = value["messages"][0]
                # print(f"Agent 생각: {msg.content}") # 디버깅용
            elif key == "tools":
                # print(f"Tool 실행 결과...") # 디버깅용
                pass
    
    # 최종 응답 출력
    final_response = value["messages"][0].content
    print(f"챗봇: {final_response}")
    return final_response

In [83]:
rag_result = chat("대구 공영장례와 화장장려금 신청 대상에 대해 알려줘.")


사용자: 대구 공영장례와 화장장려금 신청 대상에 대해 알려줘.
Tool Calls 존재 여부: [{'name': 'search_public_funeral_ordinance', 'args': {'query': '신청 대상', 'region': '대구광역시'}, 'id': 'call_XOieZmqmCccRIDv9lowplMf7', 'type': 'tool_call'}, {'name': 'search_cremation_subsidy_ordinance', 'args': {'query': '신청 대상', 'region': '대구광역시'}, 'id': 'call_TrHQf53Q51loYTx1tTm6ApIO', 'type': 'tool_call'}]
공영 장례 조례 매칭된 지역들 최대 3개 ['인천광역시']
화장 장려금 조례 매칭된 지역들 최대 3개 ['대구광역시']
툴 검색 결과: [Document(id='02297fa4-e8da-4bec-afbb-985bbc9fa86e', metadata={'region': '인천광역시', 'type': 'Public_Funeral_Ordinance'}, page_content='(현행, 제정) 2020.03.30. 제6367호\n인천광역시 공영장례 지원 조례\n지원대상\n시장은 무연고 및 저소득층 사망자로서 사망 당시 시에 주민등록을 둔 사망자인 경우 다음 어느 하나에\n해당하는 경우 장례를 지원할 수 있다.\n1. 「장사 등에 관한 법률」 제12조제1항의 무연고 사망자\n2. 「국민기초생활보장법」, 「긴급복지지원법」 및 「인천광역시 저소득주민의 생활안정 지원 조례」에 따라\n장제급여를 받는 수급자로서 규칙으로 정하는 자\n3. 연고자가 없거나, 연고자를 알 수 없거나, 연고자가 있으나 시신 인수를 거부ㆍ기피하는 등의 사람\n으로서 규칙으로 정하는 자\n4. 그 밖에 공영장례 지원이 필요하다고 시장이 인정하는 경우\n지원내용\n1. 시장은 지원 대상자가 사망한 경우 예산의 범위에서 공영장례에 필요한 행정적·재정적 지원을 할\n수 있

In [51]:
rag_result = chat("대구 화장 시설과 묘지, 화장 장려금, 공영 장례에 대해 알려줘")


사용자: 대구 화장 시설과 묘지, 화장 장려금, 공영 장례에 대해 알려줘
Tool Calls 존재 여부: [{'name': 'search_funeral_facilities', 'args': {'query': ['화장장', '묘지'], 'region': '대구'}, 'id': 'call_xPpWubJB7G7cIDTPDyjYIcGB', 'type': 'tool_call'}, {'name': 'search_cremation_subsidy_ordinance', 'args': {'query': '화장 장려금', 'region': '대구'}, 'id': 'call_sBh1odQUncc3PpKZewf3uv59', 'type': 'tool_call'}, {'name': 'search_public_funeral_ordinance', 'args': {'query': '공영 장례', 'region': '대구'}, 'id': 'call_pDb1cyEwA4MMHFO9k1yOfVrd', 'type': 'tool_call'}]
입력된 쿼리 리스트: ['화장장', '묘지']
[crematorium] 매칭된 지역: ['대구광역시 수성구']
화장 장려금 조례 매칭된 지역들 최대 3개 ['대구광역시']
공영 장례 조례 매칭된 지역들 최대 3개 None
[cemetery] 매칭된 지역: ['대구광역시 군위군', '대구광역시 달서구']
툴 검색 결과: [Document(id='afbfb485-bc19-414b-83b0-066408e664fb', metadata={'region': '대구광역시', 'sub_type': 'cremation_etcetera', 'type': 'Cremation_Subsidy_Ordinance'}, page_content='「대구광역시 장사시설의 설치 및 운영 조례」 제6조의2\n● 시장은 관내 화장시설의 공사 및 재난사고 등으로 가동이 중지되어 사용 불가능한\n 경우, 예약이 완료되어 사용 불가능한 경우 예산의 범위 내에서 화장시설 사용료를\n 지원할 수 있다. ● 

In [169]:
rag_result = chat("포천시에서 천주교 관련 묘지 찾아줘.")


사용자: 포천시에서 천주교 관련 묘지 찾아줘.
Tool Calls 존재 여부: [{'name': 'search_funeral_facilities', 'args': {'query': '천주교 묘지', 'region': '포천시'}, 'id': 'call_w81WHeqoJeG2mkesilxhsGW6', 'type': 'tool_call'}]
쿼리 : 천주교 묘지, 지역 : 포천시
매칭된 지역: ['경기도 포천시']
검색 결과 8건 반환
Tool Calls 존재 여부: []
챗봇: 포천시에서 천주교와 관련된 묘지를 찾으셨군요. 아래 두 곳을 추천드립니다:

1. **천주교혜화동성당 포천묘원**
   - 주소: 경기도 포천시 소흘읍 윗용상길 68 (이동교리)
   - 전화번호: 031-542-1410

2. **금호동성당 천보묘원**
   - 주소: 경기도 포천시 화합로 116 (동교동)
   - 전화번호: 031-541-0098

이 두 곳은 천주교(가톨릭)와 관련된 시설입니다. 필요한 정보가 도움이 되길 바라며, 언제든지 추가적인 도움이 필요하시면 말씀해 주세요.


In [173]:
print(rag_result)

포천시에서 천주교와 관련된 묘지를 찾으셨군요. 아래 두 곳을 추천드립니다:

1. **천주교혜화동성당 포천묘원**
   - 주소: 경기도 포천시 소흘읍 윗용상길 68 (이동교리)
   - 전화번호: 031-542-1410

2. **금호동성당 천보묘원**
   - 주소: 경기도 포천시 화합로 116 (동교동)
   - 전화번호: 031-541-0098

이 두 곳은 천주교(가톨릭)와 관련된 시설입니다. 필요한 정보가 도움이 되길 바라며, 언제든지 추가적인 도움이 필요하시면 말씀해 주세요.


In [188]:
rag_result = chat("경남 장례식장을 알려줘.")


사용자: 경남 장례식장을 알려줘.
Tool Calls 존재 여부: [{'name': 'search_funeral_facilities', 'args': {'query': '장례식장', 'region': '경남'}, 'id': 'call_XkFbZO2RCF7C2zafwAVenbJG', 'type': 'tool_call'}]
쿼리 : 장례식장, 지역 : 경남, 지역들 : None
매칭된 지역: ['경남 고성군', '경남 창원시']
검색 결과 2건 반환
최종 병합된 검색 결과: 2건
Tool Calls 존재 여부: []
챗봇: 경남 지역의 장례식장 정보를 아래에 안내해 드립니다:

1. **고성군공설화장장**
   - 주소: 경남 고성군 상리면 자은리 산85-1
   - 전화번호: 055-670-2923
   - 유형: 화장시설

2. **창원시립마산화장장**
   - 주소: 경남 창원시 마산합포구 진동면 공원묘원로 232
   - 전화번호: 055-712-0224
   - 유형: 화장시설

필요한 추가 정보가 있거나 다른 지역의 시설이 필요하시면 언제든지 말씀해 주세요.


In [205]:
rag_result = chat("서울 근처 경기도쪽 장례식장을 알려줘.")


사용자: 서울 근처 경기도쪽 장례식장을 알려줘.
Tool Calls 존재 여부: [{'name': 'search_funeral_facilities', 'args': {'query': '장례식장', 'regions': ['경기도 고양시', '경기도 성남시', '경기도 부천시']}, 'id': 'call_PDuZNJL5mPnLG92f5gaB776Z', 'type': 'tool_call'}]
쿼리 : 장례식장, 지역 : None, 지역들 : ['경기도 고양시', '경기도 성남시', '경기도 부천시']
매칭된 지역: ['경기도 고양시']
검색 결과 3건 반환
매칭된 지역: ['경기도 성남시']
검색 결과 3건 반환
매칭된 지역: ['경기도 부천시']
검색 결과 3건 반환
최종 병합된 검색 결과: 9건
Tool Calls 존재 여부: []
챗봇: 서울 근처 경기도 지역의 장례식장을 몇 군데 소개해드릴게요:

### 경기도 고양시
1. **동국대학교일산장례식장**
   - 주소: 경기도 고양시 일산동구 동국로 27 (식사동, 동국대학교일산병원)
   - 전화번호: 031-961-9400

2. **베스트장례식장**
   - 주소: 경기도 고양시 일산서구 경의로 816-1 (덕이동)
   - 전화번호: 031-924-4444

3. **일산복음병원장례식장**
   - 주소: 경기도 고양시 일산동구 고양대로 760, 나동 B1층 (중산동, 일산복음병원)
   - 전화번호: 031-977-6000

### 경기도 성남시
1. **성남시의료원장례식장**
   - 주소: 경기도 성남시 수정구 수정로171번길 10 (태평동, 성남시의료원)
   - 전화번호: 031-738-7450

2. **주식회사 분당제생병원장례식장**
   - 주소: 경기도 성남시 분당구 서현로180번길 20 (서현동, 분당제생병원)
   - 전화번호: 031-708-4444

3. **성남중앙병원장례식장**
   - 주소: 경기도 성남시 중원구 산성대로476번길 12 (금광동)
   - 전화번호: 031-

In [ ]:
# # 케이스 2: 암시적인 상황 (도구 호출 전 지역 확인 유도 예상)
rag_result = chat("갑자기 아버지가 돌아가셨어.. 어떡해야 할지 모르겠네.")


사용자: 갑자기 아버지가 돌아가셨어.. 어떡해야 할지 모르겠네.
Tool Calls 존재 여부: []
챗봇: 먼저, 깊은 위로의 말씀을 드립니다. 이런 상황에서 무엇을 해야 할지 막막하실 수 있습니다. 제가 도와드릴 수 있는 부분이 있다면 최대한 지원해드리겠습니다.

장례 절차나 시설에 대한 정보가 필요하시면 말씀해 주세요. 어느 지역에서 장례를 준비하실 계획인지 알려주시면, 그 지역에 맞는 정보를 찾아드리겠습니다.


In [54]:
# # 케이스 3: 암시적 상황 + 지역 포함
rag_result = chat("지금 부산 해운대 쪽인데, 급하게 빈소를 알아봐야 해.")


사용자: 지금 부산 해운대 쪽인데, 급하게 빈소를 알아봐야 해.
Tool Calls 존재 여부: [{'name': 'search_funeral_facilities', 'args': {'query': ['장례식장'], 'region': '부산 해운대구'}, 'id': 'call_8k9Dthl1HuSDiGO1ZPeNPa7S', 'type': 'tool_call'}]
입력된 쿼리 리스트: ['장례식장']
[funeral_home] 매칭된 지역: ['부산광역시 해운대구']
Tool Calls 존재 여부: []
챗봇: 부산 해운대구에서 빈소를 찾으신다면, 다음의 장례식장을 고려해 보실 수 있습니다. 힘든 시기에 조금이나마 도움이 되길 바랍니다.

1. **반송장례식장**
   - 주소: 부산광역시 해운대구 반송로 832 (반송동)
   - 전화번호: 051-525-1024

2. **성심장례식장**
   - 주소: 부산광역시 해운대구 좌동순환로 462 (중동, 성심병원)
   - 전화번호: 051-747-5600

필요하신 정보가 더 있다면 언제든지 말씀해 주세요.
